# Contrastive Learning Lab Binôme_8
### Objectif du Lab
Ce notebook explore le Contrastive Learning, une technique d'apprentissage auto-supervisé qui vise à apprendre des représentations de données en rapprochant les exemples similaires et en éloignant les exemples différents.

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
import torchvision
import torchvision.transforms as transforms

# Définition des transformations pour prétraiter les images
transform = transforms.Compose([
    transforms.RandomResizedCrop(32),  # Recadrage aléatoire
    transforms.RandomHorizontalFlip(),  # Flip horizontal aléatoire
    transforms.ToTensor(),  # Conversion en tenseur PyTorch
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalisation des pixels
])

# Chargement du dataset CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Création des DataLoaders pour optimiser l'entraînement
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

print("Données CIFAR-10 chargées avec succès !")

Données CIFAR-10 chargées avec succès !


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Définition du modèle Contrastive Learning avec des noms en français
class ModeleContrastif(nn.Module):
    def __init__(self, dimension_embedding=128):
        super(ModeleContrastif, self).__init__()
        self.extracteur_caracteristiques = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(8192, dimension_embedding),
        )

    def forward(self, images):
        return F.normalize(self.extracteur_caracteristiques(images), p=2, dim=1)  # Normalisation L2

# Instanciation du modèle
modele_contrastif = ModeleContrastif()
print("Modèle Contrastif défini avec succès !")

Modèle Contrastif défini avec succès !


In [ ]:
class PerteContrastive(nn.Module):
    def __init__(self, marge=1.0):
        super(PerteContrastive, self).__init__()
        self.marge = marge

    def forward(self, embedding1, embedding2, label):
        distance = F.pairwise_distance(embedding1, embedding2)
        perte = label * torch.pow(distance, 2) + (1 - label) * torch.pow(torch.clamp(self.marge - distance, min=0.0), 2)
        return perte.mean()

# Instanciation de la fonction de perte
fonction_perte = PerteContrastive()
print("Fonction de perte contrastive définie avec succès !")

Fonction de perte contrastive définie avec succès !


In [ ]:
import torch.optim as optim

# Définition de l'optimiseur
optimiseur = optim.Adam(modele_contrastif.parameters(), lr=0.001)

# Boucle d'entraînement avec arrêt automatique + gestion des interruptions
def entrainer_modele(epochs=10, seuil_perte=0.01, checkpoint_interval=5):
    try:
        for epoch in range(epochs):
            perte_totale = 0.0

            print(f"Début de l'entraînement - Époque {epoch+1}/{epochs}")

            for i, (images, labels) in enumerate(train_loader):
                images = images.to(torch.device("cpu"))

                if images.size(0) < 2:
                    print(f" Lot {i}: Pas assez d'images pour créer des paires, on ignore cette itération.")
                    continue

                # Création des paires d'images
                image1, image2 = images[:64], images[64:]
                labels_paires = (labels[:64] == labels[64:]).float()

                # Passage des images à travers le modèle
                embedding1 = modele_contrastif(image1)
                embedding2 = modele_contrastif(image2)

                # Calcul de la perte contrastive
                perte = fonction_perte(embedding1, embedding2, labels_paires)

                # Optimisation
                optimiseur.zero_grad()
                perte.backward()
                optimiseur.step()

                perte_totale += perte.item()

                # Affichage après chaque lot pour suivre la progression
                print(f" Lot {i+1}/{len(train_loader)} - Perte courante : {perte.item():.4f}")

            # Vérification pour arrêt automatique
            print(f" Époque {epoch+1}/{epochs} terminée - Perte totale : {perte_totale:.4f}", flush=True)

            if perte_totale < seuil_perte:
                print(f" Arrêt automatique activé : perte ({perte_totale:.4f}) inférieure au seuil ({seuil_perte}) !")
                break

            # Sauvegarde automatique du modèle tous les {checkpoint_interval} epochs
            if (epoch + 1) % checkpoint_interval == 0:
                torch.save(modele_contrastif.state_dict(), f"checkpoint_epoch_{epoch+1}.pth")
                print(f"Checkpoint sauvegardé à l'époque {epoch+1} !")

        print(" Entraînement terminé avec succès !")

    except KeyboardInterrupt:
        print("Entraînement interrompu manuellement. Sauvegarde du modèle en cours...")
        torch.save(modele_contrastif.state_dict(), "checkpoint_interruption.pth")
        print("Modèle sauvegardé avant l'interruption. Relance possible depuis le checkpoint !")


# Charger le modèle depuis le dernier checkpoint avant de reprendre l'entraînement
try:
    modele_contrastif.load_state_dict(torch.load("checkpoint_interruption.pth"))
    print("Modèle rechargé depuis le dernier checkpoint !")
except FileNotFoundError:
    print("Aucun checkpoint précédent trouvé. Début d'un entraînement complet.")

#  Relancer l'entraînement à partir du checkpoint ou depuis zéro
entrainer_modele(epochs=10, seuil_perte=0.01, checkpoint_interval=5)

Modèle rechargé depuis le dernier checkpoint !
Début de l'entraînement - Époque 1/10
 Lot 1/391 - Perte courante : 0.0905
 Lot 2/391 - Perte courante : 0.1490
 Lot 3/391 - Perte courante : 0.0900
 Lot 4/391 - Perte courante : 0.0778
 Lot 5/391 - Perte courante : 0.1328
 Lot 6/391 - Perte courante : 0.1239
 Lot 7/391 - Perte courante : 0.0971
 Lot 8/391 - Perte courante : 0.1311
 Lot 9/391 - Perte courante : 0.1083
 Lot 10/391 - Perte courante : 0.0909
Entraînement interrompu manuellement. Sauvegarde du modèle en cours...
Modèle sauvegardé avant l'interruption. Relance possible depuis le checkpoint !


In [ ]:
# Importer tqdm (pour afficher la barre de progression)
from tqdm import tqdm

def evaluer_modele():
    total_correct = 0
    total_samples = 0
    print("Début de l'évaluation du modèle...")

    with torch.no_grad():  # Désactivation du calcul du gradient
        for images, labels in tqdm(test_loader, desc="Évaluation en cours"):
            images = images.to(torch.device("cpu"))

            # Calculer la taille du batch actuel
            batch_size = images.size(0)
            moitié_batch = batch_size // 2  # Diviser dynamiquement

            if moitié_batch == 0:
                continue

            # Création des paires de taille adaptative
            image1, image2 = images[:moitié_batch], images[moitié_batch:]
            labels_paires = (labels[:moitié_batch] == labels[moitié_batch:]).float()

            # Passage des images dans le modèle
            embedding1 = modele_contrastif(image1)
            embedding2 = modele_contrastif(image2)

            # Calcul de la distance entre les embeddings
            distance = torch.norm(embedding1 - embedding2, p=2, dim=1)

            # Prédiction : si distance faible → images similaires
            predictions = (distance < 0.5).float()

            # Calcul du nombre de prédictions correctes
            total_correct += (predictions == labels_paires).sum().item()
            total_samples += labels_paires.size(0)

        # Calcul et affichage de la précision
        if total_samples > 0:
            precision = total_correct / total_samples * 100
            print(f"Précision du modèle : {precision:.2f}%", flush=True)
        else:
            print("Impossible de calculer la précision (pas assez d'échantillons).")

# Lancer l'évaluation
evaluer_modele()

Début de l'évaluation du modèle...


Évaluation en cours: 100%|████████████████████| 79/79 [00:40<00:00,  1.96it/s]

Précision du modèle : 89.74%
